<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./2_2_SRAL_advanced_data_access_hda.ipynb">SRAL advanced data access; WEkEO HDA >></a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h3>Learn SRAL: Advanced</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
The following modules are prerequisites for this notebook:
- **<a href="../1_SRAL_introductory/1_1_SRAL_data_access_HDA_Data_Store.ipynb">1_1_SRAL_data_access_HDA_Data_Store</a>**

</div>
<hr>

# 2.1 SRAL advanced data access via the Data Store using eumdac

### Data used

| Product Description  | Data Store collection ID| Product Navigator |
|:--------------------:|:-----------------------:|:-----------------:|
| Sentinel-3 SRAL level-2 | EO:EUM:DAT:0415 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:SR_2_WAT___NTC?query=SRAL&s=advanced" target="_blank">link</a> |

### Learning outcomes

At the end of this notebook you will know;
* How to refine your <font color="#138D75">**searches**</font> for SRAL products in the EUMETSAT Data Store using the `eumdac client`
* How to <font color="#138D75">**download**</font> components of products

### Outline

The EUMETSAT Data Store offers many ways to interact with data in order to refine searches. Many of these methods are supported by the EUMETSAT Data Access Client (`eumdac`). In this notebook we will showcase some of the possibilities for using `eumdac` to better interact with SRAL collections. This notebook assumes that you already have an understanding of the available Data Store interfaces, which you can gain by running the **<a href="../1_SRAL_introductory/1_1_SRAL_data_access_HDA_Data_Store.ipynb">1_1_SRAL_data_access_HDA_Data_Store</a>** notebook.


<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
1. [Example 1: Filter by collections](#section1)
1. [Example 2: Filter by time](#section2)
1. [Example 3: Filter by space and time](#section3)
1. [Example 4: Download by component](#section4)

<hr>

In [2]:
import os
import json
import datetime
import shutil
import eumdac
import inspect
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import eumartools
from shapely import geometry, vectorized

# Create a download directory for our SRAL products
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

Before we use the Data Store to download data, we must first authenticate our access and retrieve an access token. More information on setting this up can be found in the **<a href="../1_SRAL_introductory/1_1_SRAL_data_access_HDA_Data_Store.ipynb">1_1_SRAL_data_access_HDA_Data_Store</a>** notebook.

In [3]:
# load credentials
with open(os.path.join(os.path.expanduser("~"),'.eumdac_credentials')) as json_file:
    credentials = json.load(json_file)
    token = eumdac.AccessToken((credentials['consumer_key'], credentials['consumer_secret']))
    print(f"This token '{token}' expires {token.expiration}")

# create data store object
datastore = eumdac.DataStore(token)



This token 'c71d2389-4c24-3786-a694-39b902cc56ed' expires 2022-02-21 22:17:35.350703


We will work with the SRAL Level-2 WAT collection throughout this notebook. The code for this is `EO:EUM:DAT:0415`

In [1]:
# set collection ID for SRAL L2 WAT
collectionID = 'EO:EUM:DAT:0415'

<div class="alert alert-info" role="alert">

## <a id='section1'></a>Example 1: Filtering by collection
[Back to top](#TOC_TOP)

</div>

To filter by collection, we simply provide the collectionID to the `datastore.get_collection method`

In [4]:
# Use collection ID
selected_collection = datastore.get_collection(collectionID)
print(selected_collection.title)

SRAL Level 2 Altimetry Global - Sentinel-3


<div class="alert alert-info" role="alert">

## <a id='section2'></a>Example 2: Filtering by time
[Back to top](#TOC_TOP)

</div>

To get the latest product in a collection, we can just ask for the `.first()` method, when searching in our selected collection using `.search()`.

In [5]:
# Get the latest product in a collection
selected_collection = datastore.get_collection(collectionID)
latest = selected_collection.search().first()
print(latest)

S3A_SR_2_WAT____20220221T182346_20220221T182854_20220221T194042_0308_082_169______MAR_O_NR_004.SEN3


To filter by time, we can pass python datetime arguments to the **dtstart** and **dtend** arguments of our collection when using the `.search()` method.

In [6]:
# time filter the collection for products
start = datetime.datetime(2022, 1, 23, 16, 0)
end = datetime.datetime(2022, 1, 23, 16, 15)
products = selected_collection.search(dtstart=start, dtend=end)

for product in products:
    print(product)

S3A_SR_2_WAT____20220123T161459_20220123T161542_20220123T183921_0043_081_140______MAR_O_NR_004.SEN3
S3B_SR_2_WAT____20220123T160644_20220123T161643_20220123T181213_0599_061_382______MAR_O_NR_004.SEN3
S3B_SR_2_WAT____20220123T155723_20220123T160643_20220123T181206_0560_061_382______MAR_O_NR_004.SEN3
S3A_SR_2_WAT____20220123T155606_20220123T160052_20220123T171909_0286_081_140______MAR_O_NR_004.SEN3
S3B_SR_2_WAT____20220123T154045_20220123T162735_20220125T075955_2810_061_382______MAR_O_ST_004.SEN3
S3B_SR_2_WAT____20220123T154045_20220123T162735_20220218T075825_2810_061_382______MAR_O_NT_004.SEN3
S3A_SR_2_WAT____20220123T153546_20220123T162014_20220125T072633_2668_081_139______MAR_O_ST_004.SEN3
S3A_SR_2_WAT____20220123T153546_20220123T162014_20220218T073230_2668_081_139______MAR_O_NT_004.SEN3


<div class="alert alert-info" role="alert">

## <a id='section3'></a>Example 3: Filtering by space and time
[Back to top](#TOC_TOP)

</div>

We can also add geographical filtering by passing in a <a href="https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry" target="_blank">Well Known Text</a> (WKT) format region of interest into the **geo** argument of the `.search()` method.

In [7]:
# space/time filter the collection for products
selected_collection = datastore.get_collection(collectionID)
start = datetime.datetime(2022, 1, 23)
end = datetime.datetime(2022, 1, 24)
roi = [[-1.0, -1.0], [4.0, -4.0], [8.0, -2.0], [9.0, 2.0], [6.0, 4.0], [1.0, 5.0], [-1.0, -1.0]]

products = selected_collection.search(
    geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in roi])),
    dtstart=start, 
    dtend=end)

for product in products:
    print(product)

S3B_SR_2_WAT____20220123T215659_20220123T220101_20220123T231152_0242_061_385______MAR_O_NR_004.SEN3
S3B_SR_2_WAT____20220123T213921_20220123T222441_20220218T134932_2720_061_385______MAR_O_NT_004.SEN3
S3B_SR_2_WAT____20220123T213921_20220123T222441_20220125T135007_2720_061_385______MAR_O_ST_004.SEN3
S3A_SR_2_WAT____20220123T095943_20220123T100601_20220123T121301_0378_081_136______MAR_O_NR_004.SEN3
S3A_SR_2_WAT____20220123T093618_20220123T102415_20220218T021835_2877_081_136______MAR_O_NT_004.SEN3
S3A_SR_2_WAT____20220123T093618_20220123T102415_20220125T020852_2877_081_136______MAR_O_ST_004.SEN3


<div class="alert alert-info" role="alert">

## <a id='section4'></a>Example 4: Downloading by component
[Back to top](#TOC_TOP)

</div>

It is possible to download any component of a SAFE format file. This can be achieved by using iterating through the `entries` attribute of a product and string matching to your component of interest - in this case the manifest file. This method can be used to prune downloads for the enhanced, reduced or standard measurements only.

*Note: It is not currently possible to download a single variable from **inside** a netCDF file*.

In [8]:
# Get the latest product in a collection
selected_collection = datastore.get_collection(collectionID)
latest = selected_collection.search().first()

for entry in latest.entries:
    if 'xfdumanifest.xml' in entry:
        with latest.open(entry=entry) as fsrc, open(os.path.join(os.getcwd(), 'products',fsrc.name),
                                                    mode='wb') as fdst:
            print(f'Downloading {fsrc.name}.')
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of file {fsrc.name} finished.')

Download of file xfdumanifest.xml finished.


<hr>
<a href="../Index.ipynb"><< Index</a>
<br>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>